# Sandbox

This is a top level notebook for developing and playing with code in the local directory.

The expected use is not generally to share code, but to have a space for notebook activity of exploratory nature.